In [49]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy.linalg as npl
from scipy.optimize import minimize
import math as math
from itertools import combinations
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import WhiteKernel
import matplotlib.lines as mlines
from sklearn.model_selection import train_test_split
import csv
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import griddata
from scipy.spatial import distance_matrix
from scipy import interpolate
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [3]:
columns = ["x","y","z","w","data","error"]
datatab = pd.read_table("Updateddata.txt",names = columns)
datatab

,x,y,z,w,data,error
0,0.1,-1.439897,0.0,0.25,-2.831628,0.561478
1,0.1,-1.308997,0.5,0.25,-3.006489,0.485720
2,0.1,-1.178097,1.0,0.25,-2.354835,1.270193
3,0.1,-1.047198,1.0,0.25,-2.974002,0.983807
4,0.1,-0.916298,1.0,0.25,-2.401510,1.254378
...,...,...,...,...,...,...
995,0.7,1.570796,-1.0,1.00,-2.287634,2.337560
996,0.8,1.570796,-1.0,1.00,-2.164392,1.612457
997,0.9,1.570796,-1.0,1.00,-2.152368,1.550488
998,1.0,-1.439897,-1.0,1.00,-2.579505,0.442328


In [5]:
x = datatab['x']
y = datatab['y']
z = datatab['z']
w = datatab['w']
data = datatab['data']
error = datatab['error']

In [7]:
# First dataset 
datam1 = datatab[0:250]
xm1 = datam1['x'].values
ym1 = datam1['y'].values
zm1 = datam1['z'].values
wm1 = datam1['w'].values
dm1 = datam1['data'].values
em1 = datam1['error'].values
X1 = np.column_stack((xm1, ym1, zm1, wm1))

# Second dataset
datam2 = datatab[250:500]
xm2 = datam2['x'].values
ym2 = datam2['y'].values
zm2 = datam2['z'].values
wm2 = datam2['w'].values
dm2 = datam2['data'].values
em2 = datam2['error'].values
X2 = np.column_stack((xm2, ym2, zm2, wm2))

# Third dataset
datam3 = datatab[500:750]
xm3 = datam3['x'].values
ym3 = datam3['y'].values
zm3 = datam3['z'].values
wm3 = datam3['w'].values
dm3 = datam3['data'].values
em3 = datam3['error'].values
X3 = np.column_stack((xm3, ym3, zm3, wm3))

# Fourth dataset
datam4 = datatab[750:]
xm4 = datam4['x'].values
ym4 = datam4['y'].values
zm4 = datam4['z'].values
wm4 = datam4['w'].values
dm4 = datam4['data'].values
em4 = datam4['error'].values
X4 = np.column_stack((xm4, ym4, zm4, wm4))

In [9]:
X = np.column_stack((x,y, z, w))
D = np.concatenate((dm1,dm2,dm3,dm4))
E = np.concatenate((em1,em2,em3,em4))
print(D.shape)
print(E.shape)
print(X.shape)

(1000,)
(1000,)
(1000, 4)


In [61]:
n_splits = 10  

gpr_list = []

# Lists to store multiple iterations of splits
Xtrain, Xtest = [], []
Dtrain, Dtest = [], []
Etrain, Etest = [], []
trainindex, testindex = [], []

for split in tqdm(range(n_splits), desc="Training Progress"):
    # Perform train-test split for each dataset
    X_train, X_test, D_train, D_test, E_train, E_test, train_index, test_index = train_test_split(
        X, D, E, range(len(X)), test_size=0.2, random_state=42 + split, shuffle=True)

    # Define the Gaussian Process kernel
    bounds_l = ((.08, 10), (1e-2, 10.0), (1e-2, 10.0), (1e-2, 10.0))  
    guess_l = (1,1,1,1)
    guess_signal_var = 1.0
    bounds_signal_var = (1e-20, 1e20)

    kernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * \
             RBF(length_scale=guess_l, length_scale_bounds=bounds_l)
    
    # Train Gaussian Process Regressor
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=E_train**2, normalize_y=True, n_restarts_optimizer=20)
    gpr.fit(X_train, D_train)
    gpr_list.append(gpr)
    Xtrain.append(X_train)
    Xtest.append(X_test)
    Dtrain.append(D_train)
    Dtest.append(D_test)
    Etrain.append(E_train)
    Etest.append(E_test)
    trainindex.append(train_index)
    testindex.append(test_index)

    # Append current split's train-test matrices for all datasets
    
    print(f"Split {split+1}: Fitted Kernel: {gpr.kernel_}")

Training Progress:  10%|█         | 1/10 [00:21<03:09, 21.07s/it]

Split 1: Fitted Kernel: 0.771**2 * RBF(length_scale=[0.272, 0.713, 0.707, 1.82])


Training Progress:  20%|██        | 2/10 [00:48<03:16, 24.57s/it]

Split 2: Fitted Kernel: 0.841**2 * RBF(length_scale=[0.305, 0.189, 0.84, 2.24])


Training Progress:  30%|███       | 3/10 [01:22<03:23, 29.05s/it]

...

Split 8: Fitted Kernel: 0.861**2 * RBF(length_scale=[0.304, 0.186, 0.797, 2.37])


Training Progress:  90%|█████████ | 9/10 [04:15<00:27, 27.86s/it]

Split 9: Fitted Kernel: 0.769**2 * RBF(length_scale=[0.248, 0.701, 0.83, 1.89])


Training Progress: 100%|██████████| 10/10 [04:40<00:00, 28.00s/it]

Split 10: Fitted Kernel: 0.853**2 * RBF(length_scale=[0.292, 0.178, 0.828, 2.36])


In [62]:
gpr1 = gpr_list[0]
gpr2 = gpr_list[1]
Dfit1, Dstd1 = gpr1.predict(X1, return_std=True)
Dfit2, Dstd2 = gpr1.predict(X2, return_std=True)
Dfit3, Dstd3 = gpr1.predict(X3, return_std=True)
Dfit4, Dstd4 = gpr1.predict(X4, return_std=True)

In [63]:
## Global relative difference
for i in range(n_splits):
    Xtestit = Xtest[i]
    x_coords = Xtestit[:,0]
    y_coords = Xtestit[:,1]
    Dtrue=Dtest[i]
    gpr = gpr_list[i]
    Dpred = gpr.predict(Xtestit)
    relative_difference = np.abs(Dtrue - Dpred) / np.abs(Dtrue)
    data = relative_difference
    mean_value = np.mean(data)
    std_value = np.std(data)
    max_value = np.max(data) 
    print(f"Split {i},Relative Error Results: Mean={mean_value:.5f},Standard Deviation={std_value:.5f},Maximum Value={max_value:.5f}")
    mse = mean_squared_error(Dtrue, Dpred)
    print(f"Split {i}, MSE: {mse:.5f}")
    r2 = r2_score(Dtrue, Dpred)
    print(f"Split {i}, R² Score: {r2:.5f}")
    lml = gpr.log_marginal_likelihood()
    print(f"Split {i}, Log Marginal Likelihood: {lml:.5f}, more")

Split 0,Relative Error Results: Mean=0.04272,Standard Deviation=0.04486,Maximum Value=0.28432
Split 0, MSE: 0.02483
Split 0, R² Score: 0.82005
Split 0, Log Marginal Likelihood: -528.34163, more
Split 1,Relative Error Results: Mean=0.03752,Standard Deviation=0.04921,Maximum Value=0.31771
Split 1, MSE: 0.02212
Split 1, R² Score: 0.80749
Split 1, Log Marginal Likelihood: -511.73292, more
Split 2,Relative Error Results: Mean=0.04728,Standard Deviation=0.04492,Maximum Value=0.25640
Split 2, MSE: 0.02642
Split 2, R² Score: 0.76124
Split 2, Log Marginal Likelihood: -498.67219, more
Split 3,Relative Error Results: Mean=0.03656,Standard Deviation=0.03723,Maximum Value=0.26097
Split 3, MSE: 0.01798
Split 3, R² Score: 0.85761
Split 3, Log Marginal Likelihood: -511.63641, more
Split 4,Relative Error Results: Mean=0.04399,Standard Deviation=0.04253,Maximum Value=0.27130
Split 4, MSE: 0.02303
Split 4, R² Score: 0.80290
Split 4, Log Marginal Likelihood: -518.14617, more
Split 5,Relative Error Results

In [75]:
print(testindex[9])

[833, 551, 782, 535, 144, 712, 585, 513, 542, 140, 220, 869, 925, 297, 312, 189, 221, 575, 630, 364, 530, 395, 492, 718, 208, 729, 164, 327, 678, 838, 457, 596, 577, 561, 942, 548, 921, 896, 336, 891, 33, 715, 151, 215, 197, 433, 705, 302, 114, 952, 858, 231, 388, 846, 647, 963, 118, 659, 39, 488, 674, 574, 832, 473, 267, 940, 422, 639, 696, 792, 23, 115, 61, 405, 200, 580, 602, 532, 128, 694, 129, 724, 58, 300, 700, 476, 16, 284, 30, 725, 966, 238, 977, 973, 666, 541, 781, 784, 342, 867, 713, 277, 372, 836, 333, 889, 122, 137, 354, 815, 65, 965, 991, 874, 746, 624, 638, 169, 213, 997, 292, 610, 482, 672, 668, 608, 893, 882, 403, 673, 111, 367, 951, 306, 967, 255, 955, 758, 607, 757, 841, 10, 909, 650, 515, 756, 434, 625, 546, 375, 751, 242, 819, 243, 279, 382, 411, 226, 249, 204, 66, 526, 816, 211, 250, 133, 85, 589, 772, 396, 926, 165, 835, 999, 851, 222, 606, 462, 74, 980, 850, 223, 761, 274, 311, 557, 885, 67, 5, 887, 14, 744, 427, 240, 716, 975, 971, 366, 479, 870]


In [91]:
data = np.array(testindex[9]) 

bins = [0, 250, 500, 750, np.max(data) + 1]  
binned_data = np.digitize(data, bins)  
binned_dict = {f"{bins[i]} - {bins[i+1]-1}": data[(binned_data == i+1)] for i in range(len(bins)-1)}

for bin_range, values in binned_dict.items():
    print(f"Bin {bin_range}: {len(values)}")  

Bin 0 - 249: 49
Bin 250 - 499: 43
Bin 500 - 749: 53
Bin 750 - 999: 55
